In [1]:
!pip install pandas

In [2]:
import torch, baukit
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from baukit import pbar
import os

In [3]:
#MODEL_NAME = "gpt2-xl"  # gpt2-xl or EleutherAI/gpt-j-6B
MODEL_NAME = "gpt2-xl"
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=False).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME)
)
baukit.set_requires_grad(False, model)

In [4]:
def compute_euclidean(x, y):
    return np.sqrt(np.sum((x-y)**2))

def get_dists(origin,all_vecs,tokens):
    dists = np.empty(len(all_vecs))
    for v in range(len(all_vecs)):
        dist = compute_euclidean(origin,all_vecs[v])
        dists[v] = (tokens[v],all_vecs[v], dist)
    return dists

In [5]:
token_csv = "../data/tokens.csv"
token_df = pd.read_csv(token_csv)
tokens = token_df['token'].values

In [6]:
npy_datapath = "../data/npy_files"
mlp_proj_layers = [n for n,_ in model.named_modules() if 'mlp.c_proj' in n]
npy_filepaths = [npy_datapath + "/" + "".join(n.split('.')[:3]) + ".npy"
                 for n in mlp_proj_layers]

In [ ]:
layer1mats = np.load(npy_filepaths[0])
dists = get_dists(layer1mats[0], layer1mats, tokens)